# Linking molecule fragments via reaction SMIRKs
- Anjali Dixit
Generate molecules containing assorted combinations of various molecular fragments or substructures (i.e. substructures containing particular SMIRKS). Here, I experiment with using reaction SMIRKS to link substructures to generate libraries of molecules.

Objective: Link molecules using specific reaction schemes for library creation.
This file will show a reductive amination using SDF files of building blocks found in the laboratory. 


In [4]:
# Import OpenEye stuff
import openeye.oechem as oechem
import openeye.oedepict as oedepict
import openeye.oemolprop as oemolprop
from IPython.display import display
import openeye.oeomega as oeomega


# Add utility function for depiction
def depict(mol, width=500, height=200):
    from IPython.display import Image
    dopt = oedepict.OEPrepareDepictionOptions()
    dopt.SetDepictOrientation( oedepict.OEDepictOrientation_Horizontal)
    oedepict.OEPrepareDepiction(mol, dopt)
    opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
    disp = oedepict.OE2DMolDisplay(mol, opts)
    ofs = oechem.oeosstream()
    oedepict.OERenderMolecule(ofs, 'png', disp)
    ofs.flush()
    return Image(data = "".join(ofs.str()))


# Try out reactions


In [5]:
# Test by linking two molecules - anything with an aldehyde (H-C=O) can react with an amine (H-N-H) to form 
# a bond betwen the C-N atoms, dropping the C=O. 
libgen = oechem.OELibraryGen("[C:1](=[O:2]).[C:3][N:4][C:7]>>[C:1][N:4][C:7]") 
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, 'C(C=O)')
libgen.SetStartingMaterial(mol, 0)
mol.Clear()
oechem.OESmilesToMol(mol, 'CN')
libgen.SetStartingMaterial(mol, 1)

mols = []
for product in libgen.GetProducts():
    print("product smiles= %s" %oechem.OEMolToSmiles(product))
    mols.append(oechem.OEMol(product))
    
# Depict result
#depict(mols[0])

## Proceed to library generation

### First, build building block libraries

In [6]:
# Build two building block libraries for linking


# Build overall set of reactants
# Load one set of building blocks: Primary amines. uncomment lines below to troubleshoot
istream = oechem.oemolistream('Enamine_Primary_Amines_30459cmpds_wSmiles.sdf')
primary_amines = []
ct = 0
for oemol in istream.GetOEGraphMols():
    # Create a smiles string using OpenEye OEChem
    #print(oechem.OEMolToSmiles(oemol))
    if oechem.OECalculateMolecularWeight(oemol) <= 200:
        primary_amines.append(oechem.OEMolToSmiles(oemol))
    elif ("Br" in oechem.OEMolToSmiles(oemol) or "Cl" in oechem.OEMolToSmiles(oemol)):
        primary_amines.append(oechem.OEMolToSmiles(oemol))
        
print(len(primary_amines))

istream.close()


24786


Raw SDF Files came from Slack Channel with provider:Patrick Fitzgerald. SDF files can also be found on Enamine's company website. The SDF files were saved to my personal Downloads folder. 

In [7]:
# Load second set of building blocks: Aldehydes. Uncomment to troubleshoot
istream = oechem.oemolistream('Enamine_Aldehydes_5798cmpds_wSmiles.sdf')
aldehydes = []

for oemol in istream.GetOEGraphMols():
    # Create a smiles string using OpenEye OEChem
    #print(oechem.OEMolToSmiles(oemol))
    if oechem.OECalculateMolecularWeight(oemol) <= 200:
        aldehydes.append(oechem.OEMolToSmiles(oemol))
    elif ("Br" in oechem.OEMolToSmiles(oemol) or "Cl" in oechem.OEMolToSmiles(oemol)):
        aldehydes.append(oechem.OEMolToSmiles(oemol))
        
print(len(aldehydes))
    
istream.close()


4224


## Next step: Begin linking our aldehydes and primary amines

### First configure our reaction

Filter according to the Rule of 3: MW <300, XlogP <= 3, HBA and HBD groups <= 3 (based on Lipinski rules). Filtering still cannot handle molecules like selentium, silicon, salts and molecules with uncharacterized LogP values for these functional groups)

In [8]:
# Start the for loop for enumeration over both sets. 
#Whatever the order of reactants is for the SMIRKS reaction 
#needs to match the order the starting materials are written in.

libgen = oechem.OELibraryGen("[C:1](=[O:2]).[N:4]([H:5])([H:6])[#6:7]>>[C:1][N:4]([H:5])[#6:7]")
libgen.SetValenceCorrection(True)


omeg = oeomega.OEOmega()
omeg.SetMaxConfs(1)
omeg.SetStrictStereo(False)

ofs1 = oechem.oemolostream('Enamine_ReductiveAminations.sdf')
ofs1.SetFormat(oechem.OEFormat_SDF)

products = []
mol = oechem.OEGraphMol()

for idx_ald, ald in enumerate(aldehydes):
    if idx_ald < 1500:
        oechem.OESmilesToMol(mol, aldehydes[idx_ald])
        libgen.SetStartingMaterial(mol, 0)
        
        for idx_amine, amine in enumerate(primary_amines):
            
            if idx_amine < 1500:
                mol.Clear()
                oechem.OESmilesToMol(mol, primary_amines[idx_amine])
                libgen.SetStartingMaterial(mol, 1)
                
                for product in libgen.GetProducts():
                    if (oechem.OECalculateMolecularWeight(product) <= 300 and oemolprop.OEGetLipinskiAcceptorCount(product)<=3 and oemolprop.OEGetLipinskiDonorCount(product)<=3 and oemolprop.OEGetRotatableBondCount(product) <= 3): #if product satisfies these conditions, then add it to the list
                        try:
                            oemolprop.OEGetXLogP(product)
                            if oemolprop.OEGetXLogP(product) <= 3:    
                                #print(oechem.OEMolToSmiles(product))
                                if not oechem.OEHasSDData(product, "Aldehyde Reactant"):
                                    oechem.OEAddSDData(product,"Aldehyde Reactant", aldehydes[idx_ald])
                                    oechem.OEAddSDData(product,"Primary Amine Reactant", primary_amines[idx_amine])
                                    oemol.SetTitle("Smiles: %s \n Aldehyde: %s, Amine: %s" % (oechem.OEMolToSmiles(product), oechem.OEGetSDData(product, "Aldehyde"),oechem.OEGetSDData(product, "Amine")) )

                                products.append(oechem.OEMol(product))
                                
                        except RuntimeError:
                            print("Error: product=",oechem.OEMolToSmiles(product))
                                                        
             #check if there is a Br or Cl in the molecule; in this case, exclude it from the MW filter.
            
                    elif (("Br" in oechem.OEMolToSmiles(product) or "Cl" in oechem.OEMolToSmiles(product)) and oemolprop.OEGetLipinskiAcceptorCount(product)<=3 and oemolprop.OEGetLipinskiDonorCount(product)<=3):
                        try:
                            oemolprop.OEGetXLogP(product)
                            if oemolprop.OEGetXLogP(product) <= 3:    
                                #print(oechem.OEMolToSmiles(product))
                                if not oechem.OEHasSDData(product, "Aldehyde Reactant"):
                                    oechem.OEAddSDData(product,"Aldehyde Reactant", aldehydes[idx_ald])
                                    oechem.OEAddSDData(product,"Primary Amine Reactant", primary_amines[idx_amine])
                                    oemol.SetTitle("Smiles: %s \n Aldehyde: %s, Amine: %s" % (oechem.OEMolToSmiles(product), oechem.OEGetSDData(product, "Aldehyde Reactant"),oechem.OEGetSDData(product, "Amine Reactant")) )

                                products.append(oechem.OEMol(product))
                        except RuntimeError:
                            print("Error: product=",oechem.OEMolToSmiles(product))       
               
                mol.Clear()
                
                if idx_amine % 1500 == 0: #match value to idx_amine contraint value
                    print(f'{idx_ald}') #count to check progress based on # aldehydes reacted
            else:
                break
    else:
        break
        
print('Filtered based on RO3\n') 
print('Number of Compounds:' + str(len(products)))

#make sure it worked and stored information 

#for product in products:
#    print('\n'+ oechem.OEMolToSmiles(product)) 
   

0
Error: product= C#CCCCCNc1cccc2c1N=[Se]=N2.Cl
1
Error: product= C#CCCCCCNc1cccc2c1N=[Se]=N2.Cl
2
Error: product= C#CCCCCCCNc1cccc2c1N=[Se]=N2.Cl
3
4
5
6
7
8
9
10
11
12
13
14
Error: product= C#Cc1ccccc1CNc2cccc3c2N=[Se]=N3.Cl
15
Error: product= C#Cc1ccsc1CNc2cccc3c2N=[Se]=N3.Cl
16
Error: product= C#Cc1ccc(cc1)CNc2cccc3c2N=[Se]=N3.Cl
17
Error: product= C#Cc1cccc(c1)CNc2cccc3c2N=[Se]=N3.Cl
18
19
Error: product= C/C(=C\c1ccc(cc1)Cl)/CNc2cccc3c2N=[Se]=N3.Cl
20
Error: product= C/C(=C\c1cc(cc(c1)F)F)/CNc2cccc3c2N=[Se]=N3.Cl
21
Error: product= C/C(=C\c1ccccc1)/CNc2cccc3c2N=[Se]=N3.Cl
22
23
Error: product= C/C=C(\C)/CNc1cccc2c1N=[Se]=N2.Cl
24
Error: product= C=C1CC(C1)CNc2cccc3c2N=[Se]=N3.Cl
25
Error: product= C=C(CNc1cccc2c1N=[Se]=N2)Br.Cl
26
27
28
Error: product= C=CC[C@@H](CNc1cccc2c1N=[Se]=N2)c3ccccc3.Cl
29
Error: product= C=CCC1(CCCC1)CNc2cccc3c2N=[Se]=N3.Cl
30
Error: product= C=CCC1(CCCCC1)CNc2cccc3c2N=[Se]=N3.Cl
31
32
Error: product= C=CCCCNc1cccc2c1N=[Se]=N2.Cl
33
Error: product= C=CC

Error: product= CC(C)(C)[Si](C)(C)OCCNc1ccc(c(c1)Cl)OCc2cccc(c2)F
Error: product= CC(C)(C)[Si](C)(C)OCCN[C@@H](c1ccccc1)c2c3ccccc3ccc2O.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNc1ncc(s1)Cc2c(cccc2Cl)Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNc1nc(cs1)c2ccc(cc2Cl)F.Br
Error: product= C[C@@H](c1c(ccc(c1Cl)F)Cl)NCCO[Si](C)(C)C(C)(C)C
Error: product= CC(C)(C)[Si](C)(C)OCCN[C@H]1CCSc2c1cc(cc2)Cl.Cl
Error: product= Cc1cc(ccc1N2CCCC2)NCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNc1nc2c(s1)CCC2.Cl
Error: product= Cc1nc(cs1)c2ccc(cc2)NCCO[Si](C)(C)C(C)(C)C.Cl.Cl
Error: product= C[C@@H](c1ccc(cc1Cl)Cl)NCCO[Si](C)(C)C(C)(C)C
Error: product= CC(C)(C)[Si](C)(C)OCCNCCCOc1ccccc1.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNc1ncc(s1)Cc2cc(cc(c2)Cl)Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNc1nc(cs1)c2cccc(c2)Cl
Error: product= CC(C)CN1CCC(CC1)NCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCNCc1ccccc1Br
Error: product= CC(C)(C)[Si](C)(C)OCCNCCc1c[nH]c2c1cc(cc2)Cl.Cl
Error: prod

Error: product= CC(C)(C)[Si](C)(C)OCCCNC[C@@H](c1ccccc1Cl)N2CCCC2
Error: product= CCCOc1ccccc1NCCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccc(cc1)OC2CCCC2.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1nc(cs1)c2cc(ccc2Cl)Cl
Error: product= Cc1cc(c(cc1Cl)OC)NCCCO[Si](C)(C)C(C)(C)C
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccc(c(c1)Cl)N(C)C
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1nc2ccc(cc2s1)Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccc(cc1I)Cl
Error: product= Cc1cccc(c1NCCCO[Si](C)(C)C(C)(C)C)Cl
Error: product= Cc1c(cc(c(n1)NCCCO[Si](C)(C)C(C)(C)C)Cl)Cl
Error: product= Cc1c(cnc(c1Cl)NCCCO[Si](C)(C)C(C)(C)C)Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccncc1Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1nc(c(s1)c2ccccc2Cl)c3ccccc3Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ncc(s1)Cc2ccc(cc2)Br
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1nc(cs1)c2ccc(cc2)Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNCCC12CC3CC(C1)CC(C3)C2.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccc(c(c1)C

Error: product= Cc1ccc(cc1)[C@H](C2CC2)NCCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNCCCO[Si](C)(C)C(C)(C)C)N1CCc2ccccc2C1.Cl.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNCC1CCCC1.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1cc(ccc1N(C)C)Br.Cl.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1cc(ccc1N(C)C)Cl.Cl.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1ccc(cc1)COCC(F)(F)F.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNCCOc1ccc(cc1)F.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNC1CC2CC1C3C2CCC3.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1cc(ccc1N2CCCC2)Cl.Cl.Cl
Error: product= Cc1ccsc1CNCCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNCCSCc1ccc(s1)Br.Cl
Error: product= CC(C)COc1ccc(cc1Cl)NCCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](C#N)NCCCO[Si](C)(C)C(C)(C)C.Cl
Error: product= CC(C)[C@H](CNCCCO[Si](C)(C)C(C)(C)C)N1CCc2ccccc2C1.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNCc1ccccc1COc2ccccc2.Cl
Error: product= CC(C)(C)[Si](C)(C)OCCCNc1cc(ccc1OC)Br
Error: product= CC(C)(C)[Si](

Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNCCO
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNCCc2ccccn2
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2cccc3c2CCCC3
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNCCCc2ccc(cc2)OC
Error: product= CC(C)(C)c1ccc(cc1)NCc2ccc(c(c2)Br)O[Si](C)(C)C(C)(C)C
Error: product= Cc1ccc(c(c1)F)NCc2ccc(c(c2)Br)O[Si](C)(C)C(C)(C)C
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2ccc(cc2)SC#N
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNC2CCC(CC2)O
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2ccc(cc2)C(F)(F)F
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2ccc(c(c2)Cl)N(C)C
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2nc3c(cc(cc3s2)F)F
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNc2nc3ccc(cc3s2)Cl
Error: product= Cc1ccc(cc1)SCCNCc2ccc(c(c2)Br)O[Si](C)(C)C(C)(C)C
Error: product= CC(C)(C)[Si](C)(C)Oc1ccc(cc1Br)CNCC2CCCOC2c3ccccc3
Error: product= CC[C@H](C)Oc1ccc(cc1)NCc2ccc(c(c2)Br)O[Si](C)(C)C(C)(C)C
Error: product= CC

...

Error: product= Cc1ccc(cc1)[C@H](c2ccccc2)NC[C@H](C)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNCCCSc1ccc(cc1)F)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1cccc(c1OC)Cl)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CNc1ccc(cc1)N2CCCC2)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNCc1ccc(c(c1)Cl)F)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CNCCOc1ccccc1F)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1cc(ccc1N2CCCCC2)Cl)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1ccc(cc1)CN2CCCC2)O[Si](C)(C)C(C)(C)C.Cl.Cl
Error: product= C[C@@H](CNCCSc1ccc(cc1)Br)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1ccc(cc1)N2CCCCCC2)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CN[C@@H](C)c1cc(c(cc1Cl)Cl)F)O[Si](C)(C)C(C)(C)C
Error: product= CC[C@@H](C12CC3CC(C1)CC(C3)C2)NC[C@H](C)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CN[C@@H](C)c1ccc(c(c1)Cl)Cl)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CN[C@H]1CCSc2c1cccc2)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNC[C@@H](c1cccc

Error: product= C[C@@H](CN[C@H](c1ccccc1)C2CCCCC2)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNC1(CCCCC1)CO)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNC(Cc1ccccc1)C(C(F)(F)F)O)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1nc(cs1)Cc2ccc(cc2)Br)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CNc1nc(cs1)Cc2ccc(cc2Cl)Cl)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CNc1c(cc(cn1)Br)I)O[Si](C)(C)C(C)(C)C
Error: product= CCC[C@@H](C#N)NC[C@H](C)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1cc(ccc1O)Br)O[Si](C)(C)C(C)(C)C
Error: product= C[C@@H](CN[C@H](C#N)c1cccc(c1)Cl)O[Si](C)(C)C(C)(C)C.Cl
Error: product= CC[C@@H](CNC[C@H](C)O[Si](C)(C)C(C)(C)C)N(C)C.Cl.Cl
Error: product= C[C@@H](CNCC1CCC1)O[Si](C)(C)C(C)(C)C.Cl
Error: product= Cc1cc(c(c(c1)OCCCNC[C@H](C)O[Si](C)(C)C(C)(C)C)C)C.Cl
Error: product= C[C@@H](CN[C@@H](C)c1ccc(cc1)SC)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNc1ccnc2c1cccc2)O[Si](C)(C)C(C)(C)C.Cl
Error: product= C[C@@H](CNCCC(C)(C)C)O[Si](C)(C)C(C)(C)C.Cl
Erro

520
Error: product= CC1([C@H](CCS1)CNc2cccc3c2N=[Se]=N3)C.Cl
521
Error: product= C[C@@]1(C[C@@H]2[C@H](C1)C2(C)C)CNc3cccc4c3N=[Se]=N4.Cl
522
Error: product= CC1([C@H]2CC=C([C@@H]1C2)CNc3cccc4c3N=[Se]=N4)C.Cl
523
524
525
526
527
Error: product= C[C@]1(CCCS1)CNc2cccc3c2N=[Se]=N3.Cl
528
529
Error: product= CC1(CC1)CNc2cccc3c2N=[Se]=N3.Cl
530
531
Error: product= CC1(CC1)CCNc2cccc3c2N=[Se]=N3.Cl
532
533
534
535
Error: product= CC1(CCC1)CNc2cccc3c2N=[Se]=N3.Cl
536
Error: product= CC1(CCCC1)CNc2cccc3c2N=[Se]=N3.Cl
537
Error: product= CC1(CCCCC1)CNc2cccc3c2N=[Se]=N3.Cl
538
Error: product= CC1(CCCCCC1)CNc2cccc3c2N=[Se]=N3.Cl
539
540
541
Error: product= CC1(CSC1)CNc2cccc3c2N=[Se]=N3.Cl
542
Error: product= CC1CCC=C(C1CNc2cccc3c2N=[Se]=N3)C.Cl
543
Error: product= CC1C=C(CCC1CNc2cccc3c2N=[Se]=N3)C.Cl
544
545
546
547
548
Error: product= CC=C(CNc1cccc2c1N=[Se]=N2)Br.Cl
549
Error: product= CC=CC=CCNc1cccc2c1N=[Se]=N2.Cl
550
Error: product= CCNc1cccc2c1N=[Se]=N2.Cl
551
Error: product= CCC#CCNc1cccc2c1N

Error: product= CCOC1=CC(=C(CC1)CNCc2c(ccs2)C)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCCSCc2ccc(s2)Br)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CN[C@@H](C)c2cccc(c2)Br)[O-].[Na+]
Error: product= CCOC1=CC(=C(CC1)CN[C@@H](C)c2ccc(s2)Cl)[O-].[Na+]
Error: product= CCOC1=CC(=C(CC1)CN[C@@H](C)Cc2ccccc2Cl)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCC(C)(C)c2ccc(cc2)Br)[O-].[Na+]
Error: product= CCOC1=CC(=C(CC1)CN[C@@H](C)Cc2ccc(cc2)C(F)(F)F)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CN[C@H](c2ccccc2)C3CCCCC3)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCC2CCC2)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CN[C@@H](C)c2ccc(cc2)SC)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCCC(C)(C)C)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCCC2CCCCC2)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNCCc2ccc3ccccc3c2)[O-].[Na+].Cl
Error: product= CCOC1=CC(=C(CC1)CNC2CCCC(C2)C)[O-].[Na+].Cl
Error: product= CC[C@@H](c1ccc(cc1)Br)NCC2=C(C=C(CC2)OCC)[O-].[Na+].Cl
Error: product= CC[

KeyboardInterrupt: 

# Troubleshooting block. Ignore if block above works. 

Propagate these changes to the loop one above

In [6]:
# Test by linking two molecules - anything with an aldehyde (H-C=O) can react with an amine (H-N-H) to form 
# a bond betwen the C-N atoms, dropping the C=O. #6 means any carbon (aromatic or alipathic) can be used.
#libgen = oechem.OELibraryGen("[C:1](=[O:2]).[#6:3][N:4]>>[C:1][N:4][#6:3]") 
libgen = oechem.OELibraryGen("[C:1](=[O:2]).[N:4]([H:5])([H:6])[#6:7]>>[C:1][N:4]([H:5])[#6:7]") 
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, 'C#CCCCC=O')
libgen.SetStartingMaterial(mol, 0)
mol.Clear()
oechem.OESmilesToMol(mol, 'Cc1cc(cc(c1)n2c(c(c(n2)C)Br)N)C')
libgen.SetStartingMaterial(mol, 1)

mols = []
for product in libgen.GetProducts():
    print("product smiles= %s" %oechem.OEMolToSmiles(product))
    mols.append(oechem.OEMol(product))

product smiles= Cc1cc(cc(c1)n2c(c(c(n2)C)Br)N[CH]CCCC#C)C


# Generate a conformer for each and write out

In [9]:
for oemol in products:
    omeg(oemol)
    oechem.OEWriteMolecule(ofs1, oemol)
ofs1.close()
    